## Installations


In [ ]:
!pip install langchain openai playwright

In [ ]:
import asyncio
import pprint
import os
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chat_models.azure_openai import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.chains import SimpleSequentialChain, SequentialChain
from langchain.memory import SimpleMemory


from dotenv import load_dotenv

load_dotenv("credentials.env")
# Remove either base_url or azure_endpoint depending on your requirements
# If you want to use base_url, comment out the azure_endpoint line
# If you want to use azure_endpoint, comment out the base_url line

## Set API keys


In [ ]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"
MODEL_NAME = "gpt-4-32k"

# Set this to `azure`
OPENAI_API_TYPE = "azure"
# The API version you want to use: set this to `2023-05-15` for the released version.
OPENAI_API_VERSION = os.environ["AZURE_OPENAI_API_VERSION"]
# The base URL for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.
OPENAI_API_BASE = os.environ["AZURE_OPENAI_ENDPOINT"]
# The API key for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.
OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]

In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [ ]:
# Usecase 1: Get predictions from existing model (LLM Primitive)
COMPLETION_TOKENS = 500

llm = AzureChatOpenAI(
    deployment_name=MODEL_NAME, temperature=0.5, max_tokens=COMPLETION_TOKENS
)
llmchain = LLMChain(llm=llm, prompt=prompt)
print(llmchain.run("bike model"))

In [ ]:
# News sites mostly have <span> tags to scrape
cnn_url = "https://www.cnn.com"
wsj_url = "https://www.wsj.com"
nyt_url = "https://www.nytimes.com/ca/"
ecommerce_url = "https://appsumo.com"
amazon_url = "https://www.amazon.ca/s?k=computers&crid=1LUXGQOD2ULFD&sprefix=%2Caps%2C94&ref=nb_sb_ss_recent_1_0_recent"

In [ ]:
urls = [cnn_url, wsj_url, nyt_url, ecommerce_url, amazon_url]

In [23]:
import asyncio
import pprint

from ai_extractor import extract
from schemas import SchemaNewsWebsites, ecommerce_schema
from scrape import ascrape_playwright

token_limit = 8000

# News sites mostly have <span> tags to scrape
cnn_url = "https://www.cnn.com"
wsj_url = "https://www.wsj.com"
nyt_url = "https://www.nytimes.com/ca/"
ecommerce_url = "https://appsumo.com"
amazon_url = "https://www.amazon.ca/s?k=computers&crid=1LUXGQOD2ULFD&sprefix=%2Caps%2C94&ref=nb_sb_ss_recent_1_0_recent"


async def scrape_with_playwright(url: str, **kwargs):
    html_content = await ascrape_playwright(url)
    print("Extracting content with LLM")
    # print(html_content)
    html_content_fits_context_window_llm = html_content[:token_limit]
    extracted_content = extract(**kwargs, content=html_content_fits_context_window_llm)
    pprint.pprint(extracted_content)


# Scrape and Extract with LLM
asyncio.run(scrape_with_playwright(url=wsj_url, schema=SchemaNewsWebsites))

Task exception was never retrieved
future: <Task finished name='Task-6' coro=<Connection.run() done, defined at c:\00-Project\Repos\MyDvlp\AI-Samples\langchain-openai-web-scrapper\.conda\Lib\site-packages\playwright\_impl\_connection.py:268> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\aykeba\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\aykeba\AppData\Local\Temp\ipykernel_10376\3468047147.py", line 28, in <module>
    asyncio.run(scrape_with_playwright(url=wsj_url, schema=SchemaNewsWebsites))
  File "C:\Users\aykeba\AppData\Roaming\Python\Python311\site-packages\nest_asyncio.py", line 31, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\aykeba\AppData\Roaming\Python\Python311\site-packages\nest_asyncio.py", line 99, in run_until_complete
    return f.result()
  

Started scraping...


NotImplementedError: 

In [19]:
kwargs = {}  # Define the kwargs variable
await scrape_with_playwright(urls[0], **kwargs)

c:\00-Project\Repos\MyDvlp\AI-Samples\langchain-openai-web-scrapper\.conda\Lib\ast.py:50: RuntimeWarning: coroutine 'scrape_playwright' was never awaited
  return compile(source, filename, mode, flags,
Task exception was never retrieved
future: <Task finished name='Task-2' coro=<Connection.run() done, defined at c:\00-Project\Repos\MyDvlp\AI-Samples\langchain-openai-web-scrapper\.conda\Lib\site-packages\playwright\_impl\_connection.py:268> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\aykeba\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\aykeba\AppData\Local\Temp\ipykernel_10376\919281970.py", line 13, in <module>
    loop.run_until_complete(scrape_playwright())
  File "C:\Users\aykeba\AppData\Roaming\Python\Python311\site-packages\nest_asyncio.py", line 99, in run_until_complete
    return f.result()
           ^^^^^^^^

Started scraping...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\aykeba\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\aykeba\AppData\Local\Temp\ipykernel_10376\919281970.py", line 13, in <module>
    loop.run_until_complete(scrape_playwright())
  File "C:\Users\aykeba\AppData\Roaming\Python\Python311\site-packages\nest_asyncio.py", line 99, in run_until_complete
    return f.result()
           ^^^^^^^^^^
  File "c:\00-Project\Repos\MyDvlp\AI-Samples\langchain-openai-web-scrapper\.conda\Lib\asyncio\futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "c:\00-Project\Repos\MyDvlp\AI-Samples\langchain-openai-web-scrapper\.conda\Lib\asyncio\tasks.py", line 267, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "C:\Users\aykeba\AppData\Local\Temp\ipykernel_10376\919281970.py", line 7, in scrape_playwrigh

NameError: name 'scrape_with_playwright' is not defined